In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hyperdrive-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project-compute"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

## Hyperparameter Tuning in Azure ML

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": choice(0.001, 0.01, 0.1, 1, 10, 100),
    "max_iter": choice(100, 500, 1000),
})

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=3,
    slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

args = ["--C", "1.0", "--max_iter", "100"]

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    arguments=args,
    compute_target=cluster,
    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [ ]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the best run's metrics
best_run_metrics = best_run.get_metrics()

# Get best run's file name
best_run.get_file_names()

# Save best model
filename = "model_1000_10.0.pkl"
best_run.download_file(name=filename, output_file_path=filename)

## Automated ML

In [ ]:
from azureml.core import Dataset

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

# Create TabularDataset using TabularDatasetFactory
ds = Dataset.Tabular.from_delimited_files(path=url)

__Clean data and convert pandas dataframe to Dataset Class__

In [ ]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
ds_cleaned = pd.concat([x, y], axis=1)

datastore = ws.get_default_datastore()

dataset = Dataset.Tabular.register_pandas_dataframe(ds_cleaned, target=datastore, name="train_data_automl")

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run

exp_automl = Experiment(ws, "automl_experiment")
automl_run = exp_automl.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

best_run, model = automl_run.get_output()

In [ ]:
import joblib

model_path = "best_model_automl.pkl"
joblib.dump(model, model_path)

__Delete compute target__

In [16]:
cluster.delete()